<a href="https://colab.research.google.com/github/Baby757/NB/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22cifar100_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras_flops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops

In [3]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [4]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [5]:
from tensorflow.keras.datasets import cifar10

In [12]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = cifar10.load_data()

print('Shape of X_train is {}'.format(X_train.shape))
print('Shape of X_val is {}'.format(X_val.shape)) 
print('Shape of y_train is {}'.format(y_train.shape))
print('Shape of y_val is {}'.format(y_val.shape))

Shape of X_train is (50000, 32, 32, 3)
Shape of X_val is (10000, 32, 32, 3)
Shape of y_train is (50000, 1)
Shape of y_val is (10000, 1)


In [13]:
from tensorflow.keras.utils import to_categorical

In [14]:
# Импорт необходимых библиотек, которые мы можем использовать, а можем и не использовать. Всегда полезно импортировать.
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout,Input, AveragePooling2D, Activation,Conv2D, MaxPooling2D, BatchNormalization,Concatenate
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [15]:
# Преобразуйте метки классов в one_hot формат
X_train = X_train.astype('float32') 
X_val = X_val.astype('float32') 
X_val = X_val / 255.0

In [16]:
# one hot encode outputs 
y_train = np_utils.to_categorical(y_train) 
y_val = np_utils.to_categorical(y_val) 
num_classes = y_val.shape[1]


In [17]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

AssertionError: ignored

In [21]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32,32,3]),
    tf.keras.layers.Conv2D(16, 3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(CLASSES),
    tf.keras.layers.Activation('softmax')
])

In [18]:
model1=Sequential()
model1.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(32,32,3),activation='relu'))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Conv2D(filters=32,kernel_size=(4,4),input_shape=(32,32,3),activation='relu'))
model1.add(MaxPool2D(pool_size=(2,2)))
model1.add(Flatten())
model1.add(Dense(256,activation='relu'))
model1.add(Dense(10,activation='softmax'))
model1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
# вычисление количества операций
flops = get_flops(model1, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


FLOPs: 7.0226e6


In [22]:
# вывод краткой информации о модели
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 29, 29, 32)        1568      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 32)        16416     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 256)               2

In [24]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model1.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model1.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=30,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
     epochs=100
) 
# Final evaluation of the model 
scores = model1.evaluate(X_val, y_val, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/10
16667/16667 [==============================] - 88s 5ms/step - loss: 1.7191 - accuracy: 0.3662 - val_loss: 3.5705 - val_accuracy: 0.0999
Epoch 2/10
16667/16667 [==============================] - 88s 5ms/step - loss: 1.7191 - accuracy: 0.3662 - val_loss: 3.5705 - val_accuracy: 0.0999
Epoch 3/10
16667/16667 [==============================] - 88s 5ms/step - loss: 1.7191 - accuracy: 0.3662 - val_loss: 3.5705 - val_accuracy: 0.0999
Epoch 4/10
16667/16667 [==============================] - 86s 5ms/step - loss: 1.7191 - accuracy: 0.3662 - val_loss: 3.5705 - val_accuracy: 0.0999
Epoch 5/10
16667/16667 [==============================] - 86s 5ms/step - loss: 1.7191 - accuracy: 0.3662 - val_loss: 3.5705 - val_accuracy: 0.0999
Epoch 6/10
15168/16667 [==========================>...] - ETA: 7s - loss: 1.7194 - accuracy: 0.3649